##Flight Data Analysis

In [ ]:
import pandas as pd
df=pd.read_csv("flight.csv")

## Outliers (Flights >19 hours) and (Distance>15,000km)

In [ ]:
import pandas as pd
df = pd.read_csv("flight.csv")
def duration_to_hours(duration_str):
    if pd.isna(duration_str):
        return None
    hours = 0
    minutes = 0
    if "hr" in duration_str:
        hours = int(duration_str.split("hr")[0])
        if "min" in duration_str:
            minutes = int(duration_str.split("hr")[1].replace("min","").strip())
    elif "min" in duration_str:
        minutes = int(duration_str.replace("min","").strip())
    return hours + minutes/60

df["duration_hours"] = df["duration_formatted"].apply(duration_to_hours)
MAX_FLIGHT_HOURS = 19
MAX_DISTANCE_KM = 15000  # adjust if needed
# Duration outliers
duration_outliers = df[df["duration_hours"] > MAX_FLIGHT_HOURS]
print(" Duration outliers (flights > 19 hours):")
print(duration_outliers)
# Distance outliers
distance_outliers = df[df["distance"] > MAX_DISTANCE_KM]
print(" Distance outliers (flights > 20,000 km):")
print(distance_outliers)
# Combined outliers (duration or distance)
combined_outliers = df[(df["duration_hours"] > MAX_FLIGHT_HOURS) | (df["distance"] > MAX_DISTANCE_KM)]
print(" Combined outliers (duration or distance):")
print(combined_outliers)

## Duplicating and Conflicting Flights

In [ ]:
import pandas as pd
df = pd.read_csv("flight.csv")
#Check for exact duplicates (flightCode + origin + destination)
exact_duplicates = df[df.duplicated(subset=["flightCode","origin","destination"], keep=False)]
print(" Exact duplicate flights (same flightCode, origin, destination):")
print(exact_duplicates)
# Same flight code but different origin or destination
grouped = df.groupby("flightCode")[["origin","destination"]].nunique()
conflicting_flights = grouped[(grouped["origin"] > 1) | (grouped["destination"] > 1)]
print(" Flight codes with different origin/destination:")
print(conflicting_flights)
# Optional: Show the actual rows for conflicting flights
conflicting_rows = df[df["flightCode"].isin(conflicting_flights.index)]
print("Actual rows with conflicting flight codes:")
print(conflicting_rows)

## Fifth Freedom Flights

In [ ]:
import pandas as pd
df = pd.read_csv("flight.csv")
# Airline : Home Country mapping
airline_country = {
    "Lufthansa": "Germany",
    "All Nippon": "Japan",
    "Air China": "China",
    "China Southern Airlines": "China",
    "Condor": "Germany",
    "United": "USA",
    "Delta Air Lines": "USA",
    "Qatar Airways": "Qatar",
    "Thai Airways": "Thailand",
    "Air Astana": "Kazakhstan",
    "Air Baltic": "Latvia",
    "Lufthansa Cargo": "Germany",
    "AeroLogic": "Germany"
}
# Airport : Country mapping
airport_country = {
    "FRA": "Germany",
    "HND": "Japan",
    "PEK": "China",
    "CAN": "China",
    "ORD": "USA",
    "ATL": "USA",
    "BKK": "Thailand",
    "DFW": "USA",
    "RIX": "Latvia",
    "JFK": "USA",
    "GYD": "Azerbaijan",
    "ALA": "Kazakhstan",
    "SKP": "North Macedonia",
    "HKT": "Thailand",
    "SZX": "China"
}
# Add country columns
df["airline_country"] = df["airlineScrapped"].map(airline_country)
df["origin_country"] = df["origin"].map(airport_country)
df["destination_country"] = df["destination"].map(airport_country)
# Fifth freedom logic
df["fifth_freedom"] = (
    (df["airline_country"] != df["origin_country"]) &
    (df["airline_country"] != df["destination_country"])
)
# Show only fifth freedom flights
fifth_freedom_flights = df[df["fifth_freedom"] == True]

print(" Fifth Freedom Flights:")
print(fifth_freedom_flights[[
    "flightCode",
    "airlineScrapped",
    "airline_country",
    "origin",
    "origin_country",
    "destination",
    "destination_country"
]])